In [1]:
import sys
import pyfftw
import time
import ROOT
from ROOT import gROOT
from ROOT import gStyle
%jsroot on

ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")

import pickle
from array import array
from pathlib import Path
import numpy as np
from scipy import signal
sys.path.append("../package/")
import lycro_0v1 as lycro
import lycro as lyplot

from scipy.signal import find_peaks, peak_widths 

gStyle.SetOptFit(0)

Welcome to JupyROOT 6.24/00


In [2]:
run = 1177; thr = 70; window = 60
pick = "../../tmp/ju_r" + str(run) + "_thr-" + str(thr) + "_wind-" + str(window) +"_v2" + ".pik"
root = "../../tmp/ro_r" + str(run) + ".root"
print("PICK file: {}".format(pick))
print("ROOT file: {}".format(root))

PICK file: ../../tmp/ju_r1170_thr-70_wind-60_v2.pik
ROOT file: ../../tmp/ro_r1170.root


In [3]:
# read ju_signals
ju_signals = []
with open(pick, "rb") as f:
    for _ in range(pickle.load(f)):
        ju_signals.append(pickle.load(f))  
print(len(ju_signals))

123046


In [4]:
pol = 1; offset = 0; aling = 0
cards = [0, 1]; 
chs = range(0, 64);
prof_sig, prof_ped = lycro.getProfs(ju_signals, window, cards, chs, pol, offset, aling)

In [7]:
pathToSave = "/mnt/d/1170/"
add = "_"
name = "profile" + add; title = str(run) +" Profile of all signals " + add[1:]
lyplot.plot(prof_sig, name, title, pathToSave, "profile", "tick", "ADCu", 0, 4096, 0, window)
name = "pedestal" + add; title = str(run) +" Profile of all pedestal mean: {} rms: {} ".format(round(prof_ped.ProjectionY().GetMean(), 2), round(prof_ped.ProjectionY().GetRMS(), 2))+ add[1:]
lyplot.plot(prof_ped, name, title, pathToSave, "profile", "tick", "ADCu", 0, 4096, 0, window)

1170 Profile of all signals             
1170 Profile of all pedestal mean: 4.91 rms: 3.32             


Info in <TH2D::SaveAs>: ROOT file /mnt/d/1170/profile_.root has been created
Info in <TCanvas::Print>: SVG file /mnt/d/1170/profile_.svg has been created
Info in <TCanvas::Print>: pdf file /mnt/d/1170/profile_.pdf has been created
Info in <TH2D::SaveAs>: ROOT file /mnt/d/1170/pedestal_.root has been created
Info in <TCanvas::Print>: SVG file /mnt/d/1170/pedestal_.svg has been created
Info in <TCanvas::Print>: pdf file /mnt/d/1170/pedestal_.pdf has been created


In [10]:
plot = prof_sig.Clone()
#plot = anomaly.Clone()
pathToSave = "/mnt/d/tmp/"
#---
iPeriod = "0"; iPos="0"
#---
gROOT.ProcessLine("cmsText = \"Cryo Lyon\";")
gROOT.ProcessLine("writeExtraText = true;") # if extra text
#gROOT.ProcessLine("writeExtraText = false;") #remove Preliminary
gROOT.ProcessLine("extraText  = \"\";") # default extra text is "Preliminary"
gROOT.ProcessLine("lumi_sqrtS = \" \";") 

#plot.GetYaxis().SetRangeUser(0, 120);

#plot.SetMarkerColor(1); plot.SetMarkerStyle(20); plot.SetMarkerSize(0.2)
#plot.SetLineColor(1); plot.GetXaxis().SetTitle("tick"); plot.GetYaxis().SetTitle("ADCu")
plot.SetLineColor(1); plot.GetXaxis().SetTitle("x"); plot.GetYaxis().SetTitle("y")
plot.SetFillStyle( 3001); plot.SetFillColor(4);

cName = "pos"; cTitle = "c"; cX = 800; cY = 600
c = ROOT.TCanvas(cName, cTitle, cX, cY)
#c.SetGrid()
#---

p1 = ROOT.TPad("p1", "", 0, 0, 1, 1);
p1.SetGrid();

# Draw the 1st TMultiGraph
p1.SetBottomMargin(0.2)
#p1.SetRightMargin(0.15)
p1.Draw();
p1.cd();
plot.Draw("colz"); 
#line.Draw();
#fit_s.Draw()
cms_lumi_str = "CMS_lumi("+cName+", "+iPeriod+", "+iPos+");"; gROOT.ProcessLine(cms_lumi_str)

c.SaveAs("/mnt/d/" + cName + ".svg");
c.SaveAs("/mnt/d/" + cName + ".pdf");
plot.SaveAs("/mnt/d/" + cName + ".root");
c.Draw()


#--- 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: pos
Info in <TCanvas::Print>: SVG file /mnt/d/pos.svg has been created
Info in <TCanvas::Print>: pdf file /mnt/d/pos.pdf has been created
Info in <TH2D::SaveAs>: ROOT file /mnt/d/pos.root has been created
